In [115]:
import pandas as pd
import random
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings(action='ignore')
import pickle

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import sklearn.svm as svm
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization
from matplotlib_venn import venn2, venn2_circles

from xgboost import plot_importance
from scipy.stats import randint
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [79]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
def seed_everything(seed):
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed) 
    np.random.seed(seed)

seed_everything(37) # Seed 고정

# Data Load

In [3]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [4]:
sub = pd.read_csv('./sample_submission.csv')

# Data Preprocessing

X

In [5]:
train_a = train_df[train_df['PRODUCT_CODE']=='A_31']
train_to = train_df[(train_df['PRODUCT_CODE']=='T_31') | (train_df['PRODUCT_CODE']=='O_31')]

In [6]:
test_a = test_df[test_df['PRODUCT_CODE']=='A_31']
test_to = test_df[(test_df['PRODUCT_CODE']=='T_31') | (test_df['PRODUCT_CODE']=='O_31')]

In [7]:
#a 상위 200 column X_938까지할지 생각중
a_top200_list = ['X_318', 'X_248', 'X_256', 'X_2728', 'X_1048', 'X_1033', 'X_1109', 'X_1743', 'X_1055', 'X_1010', 'X_189', 'X_1774', 'X_1176', 'X_1158', 'X_1120', 'X_2190', 'X_1373', 'X_1026', 'X_2427', 'X_2423', 'X_1569', 'X_1991', 'X_1562', 'X_1037', 'X_1679', 'X_1563', 'X_1953', 'X_1720', 'X_996', 'X_2541', 'X_1329', 'X_1082', 'X_2797', 'X_1548', 'X_1665', 'X_1124', 'X_959', 'X_1422', 'X_1058', 'X_1806', 'X_1716', 'X_1408', 'X_1286', 'X_1812', 'X_1243', 'X_1231', 'X_1214', 'X_1054', 'X_993', 'X_1089', 'X_130', 'X_1211', 'X_1099', 'X_1076', 'X_2097', 'X_1945', 'X_1854', 'X_1786', 'X_1521', 'X_1356', 'X_1337', 'X_1123', 'X_1110', 'X_1053', 'X_2084', 'X_1291', 'X_141', 'X_1864', 'X_1275', 'X_1179', 'X_1948', 'X_1420', 'X_1264', 'X_1560', 'X_1543', 'X_1343', 'X_1114', 'X_1108', 'X_1083', 'X_947', 'X_2017', 'X_1856', 'X_1279', 'X_1207', 'X_967', 'X_257', 'X_251', 'X_2791', 'X_1382', 'X_1371', 'X_1190', 'X_1154', 'X_1030', 'X_973', 'X_953', 'X_2021', 'X_1954', 'X_1860', 'X_1625', 'X_1212', 'X_1194', 'X_1014', 'X_961', 'X_943', 'X_1949', 'X_1816', 'X_1668', 'X_1220', 'X_999', 'X_982', 'X_258', 'X_1794', 'X_1651', 'X_1616', 'X_1389', 'X_1344', 'X_1239', 'X_1107', 'X_1013', 'X_948', 'X_307', 'X_2700', 'X_1750', 'X_1530', 'X_1424', 'X_1423', 'X_1374', 'X_1366', 'X_1347', 'X_1160', 'X_1129', 'X_984', 'X_945', 'X_938', 'X_1406', 'X_1339', 'X_1294', 'X_1188', 'X_1029','X_979', 'X_348', 'X_290', 'X_2704', 'X_1631', 'X_1568', 'X_1534', 'X_1532', 'X_1518', 'X_1411', 'X_1369', 'X_1355', 'X_1285', 'X_1262', 'X_1205', 'X_1098', 'X_1052', 'X_972', 'X_942', 'X_345', 'X_217', 'X_2446', 'X_1919', 'X_1866', 'X_1526', 'X_1359', 'X_1357', 'X_1288', 'X_1283', 'X_1265', 'X_1202', 'X_1184', 'X_1155', 'X_1121', 'X_1101', 'X_1094', 'X_1077', 'X_1063', 'X_956', 'X_955', 'X_364', 'X_339', 'X_2418', 'X_2018', 'X_1973', 'X_1952', 'X_1635', 'X_1587', 'X_1584', 'X_1428', 'X_1425', 'X_1379', 'X_1338', 'X_1221', 'X_1197', 'X_1125', 'X_1122', 'X_128', 'X_2710', 'X_2068', 'X_2051']
len(a_top200_list)

200

In [82]:
#a 상위 200 column 0.5까지만(0.5 조건 x)
to_top182_list = ['X_932', 'X_458', 'X_73', 'X_516', 'X_718', 'X_838', 'X_401', 'X_124', 'X_783', 'X_698', 'X_493', 'X_497', 'X_422', 'X_536', 'X_556', 'X_790', 'X_419', 'X_712', 'X_490', 'X_399', 'X_442', 'X_397', 'X_613', 'X_835', 'X_700', 'X_678', 'X_709', 'X_611', 'X_546', 'X_769', 'X_659', 'X_606', 'X_436', 'X_554', 'X_484', 'X_483', 'X_126', 'X_12', 'X_498', 'X_102', 'X_780', 'X_786', 'X_687', 'X_614', 'X_485', 'X_421', 'X_489', 'X_400', 'X_774', 'X_491', 'X_663', 'X_446', 'X_398', 'X_394', 'X_841', 'X_500', 'X_462', 'X_661', 'X_90', 'X_590', 'X_385', 'X_792', 'X_714', 'X_665', 'X_608', 'X_572', 'X_456', 'X_896', 'X_816', 'X_517', 'X_897', 'X_772', 'X_530', 'X_515', 'X_482', 'X_514', 'X_496', 'X_2', 'X_895', 'X_757', 'X_560', 'X_543', 'X_501', 'X_907', 'X_894', 'X_605', 'X_581', 'X_566', 'X_473', 'X_448', 'X_105', 'X_855', 'X_716', 'X_107', 'X_850', 'X_662', 'X_595', 'X_574', 'X_573', 'X_555', 'X_460', 'X_439', 'X_438', 'X_616', 'X_599', 'X_563', 'X_452', 'X_407', 'X_589', 'X_523', 'X_461', 'X_690', 'X_539', 'X_831', 'X_789', 'X_588', 'X_454', 'X_404', 'X_388', 'X_120', 'X_848', 'X_711', 'X_671', 'X_619', 'X_596', 'X_513', 'X_451', 'X_389', 'X_127', 'X_118', 'X_815', 'X_812', 'X_766', 'X_471', 'X_464', 'X_437', 'X_101', 'X_845', 'X_763', 'X_688', 'X_677', 'X_447', 'X_413', 'X_402', 'X_883', 'X_851', 'X_791', 'X_669', 'X_562', 'X_494', 'X_468', 'X_457', 'X_412', 'X_109', 'X_16', 'X_796', 'X_794', 'X_681', 'X_668', 'X_477', 'X_98', 'X_46', 'X_18', 'X_580', 'X_570', 'X_553', 'X_499', 'X_453', 'X_440', 'X_387', 'X_45', 'X_813', 'X_775', 'X_615', 'X_607', 'X_576', 'X_537', 'X_524', 'X_476', 'X_475', 'X_403', 'X_17']
len(to_top182_list)

182

In [8]:
print(len(train_a)) #196 245
print(len(train_to))

249
349


In [9]:
print(len(test_a)) #196 245
print(len(test_to))

67
243


In [80]:
train_a.groupby(['LINE', 'Y_Class'])[[i for i in a_top200_list]].count()

X_318  X_248  X_256  X_2728  X_1048  X_1033  X_1109  X_1743  \
LINE    Y_Class                                                                
T010305 0           13     13     13       0      13      13      13       0   
        1           35     35     35       0      35      35      35       0   
        2           11     11     11       0      11      11      11       0   
T010306 0            6      6      6       0       6       6       6       0   
        1           37     37     37       0      37      37      37       0   
        2           27     27     27       0      27      27      27       0   
T050304 0            0      0      0      30      30      30      30      30   
        1            0      0      0      37      37      37      37      37   
        2            0      0      0      11      11      11      11      11   
T050307 0            0      0      0      11      11      11      11      11   
        1            0      0      0       9       9       9       9       9   
        2            0      0      0      22      22      22      22      22   

                 X_1055  X_1010  X_189  X_1774  X_1176  X_1158  X_1120  \
LINE    Y_Class                                                          
T010305 0            13       0      0       0      13      13      13   
        1            35       0      0       0      35      35      35   
        2            11       0      0       0      11      11      11   
T010306 0             6       0      0       0       6       6       6   
        1            37       0      0       0      37      37      37   
        2            27       0      0       0      27      27      27   
T050304 0            30      30     30      30      30      30      30   
        1            37      37     37      37      37      37      37   
        2            11      11     11      11      11      11      11   
T050307 0            11      11     11      11      11      11      11   
        1             9       9      9       9       9       9       9   
        2            22      22     22      22      22      22      22   

                 X_2190  X_1373  X_1026  X_2427  X_2423  X_1569  X_1991  \
LINE    Y_Class                                                           
T010305 0             0      13      13       0       0      13       0   
        1             0      35      35       0       0      35       0   
        2             0      11      11       0       0      11       0   
T010306 0             0       6       6       0       0       6       0   
        1             0      37      37       0       0      37       0   
        2             0      27      27       0       0      27       0   
T050304 0            30      30      30      30      30      30      30   
        1            37      37      37      37      37      37      37   
        2            11      11      11      11      11      11      11   
T050307 0            11      11      11      11      11      11       0   
        1             9       9       9       9       9       9       9   
        2            22      22      22      22      22      22      22   

                 X_1562  X_1037  X_1679  X_1563  X_1953  X_1720  X_996  \
LINE    Y_Class                                                          
T010305 0            13      13      13      13       0       0     13   
        1            35      35      35      35       0       0     35   
        2            11      11      11      11       0       0     11   
T010306 0             6       6       6       6       0       0      6   
        1            37      37      37      37       0       0     37   
        2            27      27      27      27       0       0     27   
T050304 0            30      30      30      30      30      30     30   
        1            37      37      37      37      37      37     37   
        2            11      11      11      11      11      11     

In [84]:
train_to.groupby(['LINE', 'Y_Class'])[[i for i in to_top182_list]].count()

X_932  X_458  X_73  X_516  X_718  X_838  X_401  X_124  X_783  \
LINE    Y_Class                                                                 
T100304 0           17     17    17     17     17     17     17     17     17   
        1          144    144   144    144    144    144    144    144    144   
        2           14     14    14     14     14     14     14     14     14   
T100306 0           11     11    11     11     11     11     11      0     11   
        1          145    145   145    145    145    145    145      0    145   
        2           18     18    18     18     18     18     18      0     18   

                 X_698  X_493  X_497  X_422  X_536  X_556  X_790  X_419  \
LINE    Y_Class                                                           
T100304 0           17     17     17     17     17     17     17     17   
        1          144    144    144    144    144    144    144    144   
        2           14     14     14     14     14     14     14     14   
T100306 0           11     11     11     11     11     11     11     11   
        1          145    145    145    145    145    145    145    145   
        2           18     18     18     18     18     18     18     18   

                 X_712  X_490  X_399  X_442  X_397  X_613  X_835  X_700  \
LINE    Y_Class                                                           
T100304 0           17     17     17     17     17     17     17     17   
        1          144    144    144    144    144    144    144    144   
        2           14     14     14     14     14     14     14     14   
T100306 0           11     11     11     11     11     11     11     11   
        1          145    145    145    145    145    145    145    145   
        2           18     18     18     18     18     18     18     18   

                 X_678  X_709  X_611  X_546  X_769  X_659  X_606  X_436  \
LINE    Y_Class                                                           
T100304 0           17     17     17     17     17     17     17     17   
        1          144    144    144    144    144    144    144    144   
        2           14     14     14     14     14     14     14     14   
T100306 0           11     11     11     11     11     11     11     11   
        1          145    145    145    145    145    145    145    145   
        2           18     18     18     18     18     18     18     18   

                 X_554  X_484  X_483  X_126  X_12  X_498  X_102  X_780  X_786  \
LINE    Y_Class                                                                 
T100304 0           17     17     17      0    17     17     17     17     17   
        1          144    144    144      0   144    144    144    144    144   
        2           14     14     14      0    14     14     14     14     14   
T100306 0           11     11     11     11    11     11     11     11     11   
        1          145    145    145    145   145    145    145    145    145   
        2           18     18     18     18    18     18     18     18     18   

                 X_687  X_614  X_485  X_421  X_489  X_400  X_774  X_491  \
LINE    Y_Class                                                           
T100304 0           17     17     17     17     17     17     17     17   
        1          144    144    144    144    144    144    144    144   
        2           14     14     14     14     14     14     14     14   
T100306 0           11     11     11     11     11     11     11     11   
        1          145    145    145    145    145    145    145    145   
        2           18     18     18     18     18     18     18     18   

                 X_663  X_446  X_398  X_394  X_841  X_500  X_462  X_661  X_90  \
LINE    Y_Class                                                                 
T100304 0           17     17     17     17     17     17     17     17    17   
        1          144    144    144    144    144    144    144    144 

In [85]:
train_df.groupby(['LINE', 'Y_Class'])[[i for i in train_df[-2875:]]].count()

PRODUCT_ID  Y_Class  Y_Quality  TIMESTAMP  LINE  \
LINE    Y_Class                                                    
T010305 0                13       13         13         13    13   
        1                35       35         35         35    35   
        2                11       11         11         11    11   
T010306 0                 6        6          6          6     6   
        1                37       37         37         37    37   
        2                27       27         27         27    27   
T050304 0                30       30         30         30    30   
        1                37       37         37         37    37   
        2                11       11         11         11    11   
T050307 0                11       11         11         11    11   
        1                 9        9          9          9     9   
        2                22       22         22         22    22   
T100304 0                17       17         17         17    17   
        1               144      144        144        144   144   
        2                14       14         14         14    14   
T100306 0                11       11         11         11    11   
        1               145      145        145        145   145   
        2                18       18         18         18    18   

                 PRODUCT_CODE  X_1  X_2  X_3  X_4  X_5  X_6  X_7  X_8  X_9  \
LINE    Y_Class                                                              
T010305 0                  13    0    0    0    0    0    0    0    0    0   
        1                  35    0    0    0    0    0    0    0    0    0   
        2                  11    0    0    0    0    0    0    0    0    0   
T010306 0                   6    0    0    0    0    0    0    0    0    0   
        1                  37    0    0    0    0    0    0    0    0    0   
        2                  27    0    0    0    0    0    0    0    0    0   
T050304 0                  30    0    0    0    0    0    0    0    0    0   
        1                  37    0    0    0    0    0    0    0    0    0   
        2                  11    0    0    0    0    0    0    0    0    0   
T050307 0                  11    0    0    0    0    0    0    0    0    0   
        1                   9    0    0    0    0    0    0    0    0    0   
        2                  22    0    0    0    0    0    0    0    0    0   
T100304 0                  17   17   17   17   17   17   17   17   17   17   
        1                 144  144  144  144  144  144  144  144  144  144   
        2                  14   14   14   14   14   14   14   14   14   14   
T100306 0                  11   11   11   11   11   11   11   11   11   11   
        1                 145  145  145  145  145  145  145  145  145  145   
        2                  18   18   18   18   18   18   18   18   18   18   

                 X_10  X_11  X_12  X_13  X_14  X_15  X_16  X_17  X_18  X_19  \
LINE    Y_Class                                                               
T010305 0           0     0     0     0     0     0     0     0     0     0   
        1           0     0     0     0     0     0     0     0     0     0   
        2           0     0     0     0     0     0     0     0     0     0   
T010306 0           0     0     0     0     0     0     0     0     0     0   
        1           0     0     0     0     0     0     0     0     0     0   
        2           0     0     0     0     0     0     0     0     0     0   
T050304 0           0     0     0     0     0     0     0     0     0     0   
        1           0     0     0     0     0     0     0     0     0     0   
        2           0     0     0     0     0     0     0     0     0     0   
T050307 0           0     0     0     0     0     0     0     0     0     0   
        1           0     0     0     0     0     0     0     0     0     0   
        2           0     0     0     0     0     0     0     0     

In [141]:
train_to.groupby('LINE')['X_92'].mean() #304라인만 측정된다면 306 결측치 174개

LINE
T100304    28.0
T100306     NaN
Name: X_92, dtype: float64

In [142]:
train_to.groupby('LINE')['X_93'].mean()#306라인만 측정된다면 304 결측치 175개

LINE
T100304     NaN
T100306    0.19
Name: X_93, dtype: float64

In [158]:
train_to.groupby('LINE')['X_529'].mean()

LINE
T100304    258.380601
T100306    257.152434
Name: X_529, dtype: float64

<train_to 분석>
X_384~399, 406~441  472~528 등등등 결측치 1개 이런걸 채워주려한다.

X529~533 처럼 두 라인에서 원래 다 측정되야 하지만, 측정되지 않은 것들을 yclass와 라인별로 median값을 넣어줄 것. 339나 262개의 결측치

In [8]:
def fill_nan(df): #미디언으로 해주는 이유는 yclass가 0인건 55 1인건 59 이런 식으로 규칙이 존재하기 때문에 함부로 평균을 내면 안될 것 같음
#     val_lst = {}
    for col in df.columns[-2875:]: #X_1 ~ X_1531 , -1345
        #모두 결측치 인 경우 채우지 않는다.
        #T304 라인만 측정된 경우, 냅두기
        #T306 라인만 측정된 경우, 냅두기

        if df[col].isnull().sum() == len(df): #모든 value가 결측치인 행
            continue
            
        elif len(df) - df[col].isnull().sum() <= 10: #너무 적은 개수만큼 측정되었다면, 그냥 채우지 않기 train_df_o의 경우 x
            continue
            
        elif df[col].isnull().sum() == 0: #모든 값이 다 측정된 경우 null 0개
            continue
            
        elif df[col].isnull().sum() < len(df): #T050이나 T010라인만 측정된 경우 어차피 값이 들어가지 않을 것임.
            nan_lst = list(df[df[col].isnull()==True].index)
            for j in nan_lst:
                yc = df['Y_Class'][j]
                line = df['LINE'][j]
                val = df[col][(df['Y_Class']==yc) & (df['LINE']==line)].median()
                df[col][j] = val
#                 val_lst[col] = val

    fill_df = df    
    return fill_df

train 데이터에만 결측치의 일부 채워준다.

In [9]:
train_fill_a = fill_nan(train_a)
train_fill_to = fill_nan(train_to)

In [223]:
train_a.groupby('LINE')['X_339', 'X_340', 'X_341'].mean()

,X_339,X_340,X_341
LINE,,,
T010305,0.000007,NaN,NaN
T010306,NaN,0.22,0.229714
T050304,NaN,NaN,NaN
T050307,NaN,NaN,NaN


In [12]:
drop_col_a = set(train_a.columns[-2875:])-set(a_top200_list)
drop_col_to = set(train_to.columns[-2875:])-set(to_top182_list)

In [13]:
train_drop_a = train_fill_a.drop(drop_col_a, axis = 1)
train_drop_to = train_fill_to.drop(drop_col_to, axis = 1)

In [14]:
test_drop_a = test_a.drop(drop_col_a, axis = 1)
test_drop_to = test_to.drop(drop_col_to, axis = 1)

In [52]:
train_drop_a

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_128,X_130,X_141,X_189,...,X_2423,X_2427,X_2446,X_2541,X_2700,X_2704,X_2710,X_2728,X_2791,X_2797
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,7813.0,NaN,567.3,8.23,...,575.0,249.90,209.992647,NaN,0.221,0.000028,0.000121,1.096296,23.074074,22.592593
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,19854.0,575.9,8.00,...,577.8,249.92,210.011429,0.199812,0.435,0.000017,0.000076,1.000000,23.100000,23.600000
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,7815.0,NaN,572.9,8.23,...,573.1,249.13,209.986765,NaN,0.221,0.000029,0.000127,1.100000,23.925926,23.370370
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,19856.0,562.2,8.00,...,577.7,249.97,209.998551,0.199941,0.435,0.000014,0.000076,1.000000,23.185185,23.592593
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,7817.0,NaN,565.9,8.23,...,575.9,249.97,210.000000,NaN,0.220,0.000029,0.000116,1.100000,23.888889,23.296296
5,TRAIN_005,2,0.537833,2022-06-13 5:55,T050307,A_31,NaN,19859.0,568.6,8.00,...,577.8,249.86,210.008571,0.200044,0.435,0.000019,0.000074,1.000000,23.407407,23.740741
6,TRAIN_006,1,0.533665,2022-06-13 6:03,T050304,A_31,7820.0,NaN,543.9,8.23,...,575.6,249.97,210.013433,NaN,0.221,0.000030,0.000119,1.100000,23.846154,23.269231
7,TRAIN_007,2,0.540003,2022-06-13 6:11,T050307,A_31,NaN,19861.0,573.3,8.00,...,580.3,249.90,210.011594,0.199221,0.435,0.000017,0.000076,1.000000,23.230769,23.615385
8,TRAIN_008,1,0.531821,2022-06-13 6:19,T050304,A_31,7822.0,NaN,571.9,8.23,...,575.5,249.95,210.013235,NaN,0.221,0.000029,0.000118,1.100000,23.888889,23.296296
9,TRAIN_009,2,0.538049,2022-06-13 6:28,T050307,A_31,NaN,19864.0,573.8,8.00,...,577.8,249.98,210.025000,0.200119,0.435,0.000015,0.000076,1.000000,23.307692,23.692308


In [107]:
train_y = train_drop_to['Y_Class']

In [101]:
#train_x = train_drop_a.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
#test_x = test_drop_a.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [108]:
train_x = train_drop_to.drop(columns = ['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality', 'LINE', 'PRODUCT_CODE'])
test_x = test_drop_to.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'LINE', 'PRODUCT_CODE'])

In [367]:
test_x

,X_2,X_12,X_16,X_17,X_18,X_45,X_46,X_73,X_90,X_98,...,X_850,X_851,X_855,X_883,X_894,X_895,X_896,X_897,X_907,X_932
0,94.0,474.4,247.5,248.0,247.2,80.5,71.2,10.12,36.0,0.00080,...,13.0,17.0,0.615484,7.493878,165.636364,165.727273,165.636364,166.454545,17.278387,13.383871
1,93.0,511.1,247.6,248.2,247.4,82.2,70.4,10.22,35.0,0.00130,...,13.0,16.0,0.609355,7.712245,166.090909,166.181818,166.545455,164.272727,17.277742,13.406452
2,95.0,511.4,247.6,247.9,247.4,82.2,70.4,10.22,35.0,0.00130,...,13.0,17.0,0.616129,7.406122,171.727273,169.363636,169.909091,169.000000,17.277097,13.370968
15,102.0,526.3,247.5,248.1,247.3,82.2,70.4,10.03,36.0,0.00140,...,13.0,17.0,0.908065,7.716327,168.545455,167.818182,168.272727,167.818182,17.276774,13.387097
16,93.0,476.8,247.5,248.0,247.3,80.5,71.2,10.19,35.0,0.00077,...,13.0,17.0,0.783548,7.432653,168.000000,167.636364,168.363636,165.090909,17.275806,13.367742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,91.0,511.9,247.5,248.1,247.3,81.5,71.0,9.85,36.0,0.00077,...,12.0,16.0,0.605161,7.595918,170.727273,172.272727,172.090909,170.090909,17.196774,13.341935
306,96.0,518.9,247.5,248.1,247.4,82.5,70.4,10.00,36.0,0.00130,...,12.0,16.0,0.618387,7.608163,168.425926,168.148148,167.407407,169.351852,17.278065,13.406452
307,91.0,511.6,247.6,248.0,247.3,81.5,71.0,9.85,36.0,0.00078,...,12.0,16.0,0.613871,7.730612,170.181818,170.454545,171.181818,170.272727,17.195806,13.361290
308,95.0,512.6,247.5,248.0,247.3,81.5,71.0,9.88,36.0,0.00077,...,12.0,16.0,0.616452,7.571429,167.454545,167.636364,165.181818,163.363636,17.278387,13.335484


In [109]:
p1 = test_x.index.tolist() #28 #39 # 
p1

[0,
 1,
 2,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 55,
 56,
 57,
 58,
 59,
 60,
 67,
 68,
 69,
 70,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,

범주형 데이터를 수치 데이터로 전환하기 위해 LabelEncoder 활용

In [103]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i]) #원래 column 값을 기준으로 fit.
    train_x[i] = le.transform(train_x[i]) #수치화, 수치로 변형

    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


# Modeling

학습용과 테스트용 데이터 세트를 위해 별도의 Dmatrix를 생성

gridsearchcv 결과

{'gamma': 1, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 1grid

In [104]:
#gridsearch로 a를 위한 파라미터 튜닝
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.2, 
                                                    stratify=train_y, 
                                                    random_state=37)


dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)
dtest_x = xgb.DMatrix(data=test_x)

params = {'max_depth' :5,
          'eta': 0.01,
          'objective':'multi:softmax',
          'num_class':3,
          'eval_metric':'mlogloss',
          'min_child_weight' : 3,
          'subsample' :1,
          'gamma' : 1 #default 0 gamma 0.2, 0.8로 하고 earlystopping 하면 score가 늘어난다.
          }
num_rounds = 1000

wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params=params, 
                      dtrain=dtrain, 
                      num_boost_round=num_rounds, 
                      early_stopping_rounds=100, 
                      evals=wlist)

[0]	train-mlogloss:1.08816	eval-mlogloss:1.09285
[1]	train-mlogloss:1.07786	eval-mlogloss:1.08748
[2]	train-mlogloss:1.06772	eval-mlogloss:1.08183
[3]	train-mlogloss:1.05773	eval-mlogloss:1.07635
[4]	train-mlogloss:1.04788	eval-mlogloss:1.07109
[5]	train-mlogloss:1.03819	eval-mlogloss:1.06571
[6]	train-mlogloss:1.02863	eval-mlogloss:1.06049
[7]	train-mlogloss:1.01922	eval-mlogloss:1.05565
[8]	train-mlogloss:1.01010	eval-mlogloss:1.04952
[9]	train-mlogloss:1.00095	eval-mlogloss:1.04471
[10]	train-mlogloss:0.99210	eval-mlogloss:1.03877
[11]	train-mlogloss:0.98320	eval-mlogloss:1.03387
[12]	train-mlogloss:0.97442	eval-mlogloss:1.02913
[13]	train-mlogloss:0.96593	eval-mlogloss:1.02346
[14]	train-mlogloss:0.95739	eval-mlogloss:1.01878
[15]	train-mlogloss:0.94897	eval-mlogloss:1.01441
[16]	train-mlogloss:0.94082	eval-mlogloss:1.00898
[17]	train-mlogloss:0.93262	eval-mlogloss:1.00451
[18]	train-mlogloss:0.92454	eval-mlogloss:0.99988
[19]	train-mlogloss:0.91671	eval-mlogloss:0.99468
[20]	train

[163]	train-mlogloss:0.31100	eval-mlogloss:0.61703
[164]	train-mlogloss:0.30898	eval-mlogloss:0.61586
[165]	train-mlogloss:0.30698	eval-mlogloss:0.61460
[166]	train-mlogloss:0.30504	eval-mlogloss:0.61330
[167]	train-mlogloss:0.30307	eval-mlogloss:0.61214
[168]	train-mlogloss:0.30116	eval-mlogloss:0.61103
[169]	train-mlogloss:0.29921	eval-mlogloss:0.60982
[170]	train-mlogloss:0.29731	eval-mlogloss:0.60829
[171]	train-mlogloss:0.29529	eval-mlogloss:0.60718
[172]	train-mlogloss:0.29341	eval-mlogloss:0.60604
[173]	train-mlogloss:0.29148	eval-mlogloss:0.60485
[174]	train-mlogloss:0.28988	eval-mlogloss:0.60357
[175]	train-mlogloss:0.28812	eval-mlogloss:0.60255
[176]	train-mlogloss:0.28625	eval-mlogloss:0.60150
[177]	train-mlogloss:0.28454	eval-mlogloss:0.60019
[178]	train-mlogloss:0.28279	eval-mlogloss:0.59879
[179]	train-mlogloss:0.28109	eval-mlogloss:0.59775
[180]	train-mlogloss:0.27929	eval-mlogloss:0.59679
[181]	train-mlogloss:0.27767	eval-mlogloss:0.59612
[182]	train-mlogloss:0.27590	ev

[324]	train-mlogloss:0.15377	eval-mlogloss:0.51444
[325]	train-mlogloss:0.15343	eval-mlogloss:0.51423
[326]	train-mlogloss:0.15310	eval-mlogloss:0.51411
[327]	train-mlogloss:0.15270	eval-mlogloss:0.51380
[328]	train-mlogloss:0.15232	eval-mlogloss:0.51333
[329]	train-mlogloss:0.15192	eval-mlogloss:0.51303
[330]	train-mlogloss:0.15147	eval-mlogloss:0.51256
[331]	train-mlogloss:0.15120	eval-mlogloss:0.51257
[332]	train-mlogloss:0.15082	eval-mlogloss:0.51231
[333]	train-mlogloss:0.15043	eval-mlogloss:0.51202
[334]	train-mlogloss:0.15011	eval-mlogloss:0.51187
[335]	train-mlogloss:0.14979	eval-mlogloss:0.51177
[336]	train-mlogloss:0.14936	eval-mlogloss:0.51133
[337]	train-mlogloss:0.14909	eval-mlogloss:0.51134
[338]	train-mlogloss:0.14877	eval-mlogloss:0.51126
[339]	train-mlogloss:0.14851	eval-mlogloss:0.51104
[340]	train-mlogloss:0.14820	eval-mlogloss:0.51096
[341]	train-mlogloss:0.14794	eval-mlogloss:0.51094
[342]	train-mlogloss:0.14757	eval-mlogloss:0.51066
[343]	train-mlogloss:0.14726	ev

In [105]:
#a 데이터의 score
print(f'best ntree limit은 {xgb_model.best_ntree_limit}')
y_pred = xgb_model.predict(dtest, ntree_limit=xgb_model.best_ntree_limit)
preds = np.round(y_pred).astype(int)
print(f1_score(y_test, preds, average = 'macro'))

best ntree limit은 357
0.7597435897435897


In [106]:
y_preds = xgb_model.predict(dtest_x, ntree_limit=xgb_model.best_ntree_limit)
preds_1 = np.round(y_preds).astype(int)
preds1 = pd.DataFrame(preds_1)
p1 = pd.Series(data = p1)
preds10 = pd.concat([preds1, p1], axis = 1)
preds10.columns = ['y_class', 'p1']
preds10.index = preds10['p1']
del preds10['p1']
preds10

,y_class
p1,
3,1
4,0
5,1
6,1
7,0
8,0
9,2
10,1
11,1


### Gridsearch

In [110]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

xgb_model = XGBClassifier(n_estimators=200)

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, 
                                                    test_size=0.2, 
                                                    stratify=train_y, 
                                                    random_state=37)

# 후보 파라미터 선정
params = {'max_depth':[6], 'min_child_weight':[1,2,3], 'colsample_bytree':[0.75,1],'learning_rate' :[0.03, 0.04]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(train_x, train_y, early_stopping_rounds=20, eval_metric='mlogloss', eval_set=[(X_val, y_val)])

#튜닝된 파라미터 출력
print(gridcv.best_params_)

[0]	validation_0-mlogloss:1.06052
[1]	validation_0-mlogloss:1.02835
[2]	validation_0-mlogloss:0.99411
[3]	validation_0-mlogloss:0.96659
[4]	validation_0-mlogloss:0.93568
[5]	validation_0-mlogloss:0.90591
[6]	validation_0-mlogloss:0.88035
[7]	validation_0-mlogloss:0.85368
[8]	validation_0-mlogloss:0.83270
[9]	validation_0-mlogloss:0.80764
[10]	validation_0-mlogloss:0.78403
[11]	validation_0-mlogloss:0.76315
[12]	validation_0-mlogloss:0.74264
[13]	validation_0-mlogloss:0.72201
[14]	validation_0-mlogloss:0.70357
[15]	validation_0-mlogloss:0.68662
[16]	validation_0-mlogloss:0.66810
[17]	validation_0-mlogloss:0.65039
[18]	validation_0-mlogloss:0.63576
[19]	validation_0-mlogloss:0.61945
[20]	validation_0-mlogloss:0.60553
[21]	validation_0-mlogloss:0.59053
[22]	validation_0-mlogloss:0.57786
[23]	validation_0-mlogloss:0.56385
[24]	validation_0-mlogloss:0.55049
[25]	validation_0-mlogloss:0.53679
[26]	validation_0-mlogloss:0.52461
[27]	validation_0-mlogloss:0.51264
[28]	validation_0-mlogloss:0.5

[32]	validation_0-mlogloss:0.41880
[33]	validation_0-mlogloss:0.40905
[34]	validation_0-mlogloss:0.39921
[35]	validation_0-mlogloss:0.39144
[36]	validation_0-mlogloss:0.38129
[37]	validation_0-mlogloss:0.37372
[38]	validation_0-mlogloss:0.36608
[39]	validation_0-mlogloss:0.35700
[40]	validation_0-mlogloss:0.34855
[41]	validation_0-mlogloss:0.34168
[42]	validation_0-mlogloss:0.33492
[43]	validation_0-mlogloss:0.32803
[44]	validation_0-mlogloss:0.31997
[45]	validation_0-mlogloss:0.31539
[46]	validation_0-mlogloss:0.30754
[47]	validation_0-mlogloss:0.30284
[48]	validation_0-mlogloss:0.29557
[49]	validation_0-mlogloss:0.29033
[50]	validation_0-mlogloss:0.28351
[51]	validation_0-mlogloss:0.27801
[52]	validation_0-mlogloss:0.27319
[53]	validation_0-mlogloss:0.26813
[54]	validation_0-mlogloss:0.26310
[55]	validation_0-mlogloss:0.25697
[56]	validation_0-mlogloss:0.25132
[57]	validation_0-mlogloss:0.24694
[58]	validation_0-mlogloss:0.24121
[59]	validation_0-mlogloss:0.23668
[60]	validation_0-ml

[64]	validation_0-mlogloss:0.24444
[65]	validation_0-mlogloss:0.23896
[66]	validation_0-mlogloss:0.23441
[67]	validation_0-mlogloss:0.23102
[68]	validation_0-mlogloss:0.22564
[69]	validation_0-mlogloss:0.22093
[70]	validation_0-mlogloss:0.21705
[71]	validation_0-mlogloss:0.21299
[72]	validation_0-mlogloss:0.20855
[73]	validation_0-mlogloss:0.20350
[74]	validation_0-mlogloss:0.19968
[75]	validation_0-mlogloss:0.19668
[76]	validation_0-mlogloss:0.19261
[77]	validation_0-mlogloss:0.19051
[78]	validation_0-mlogloss:0.18710
[79]	validation_0-mlogloss:0.18432
[80]	validation_0-mlogloss:0.18042
[81]	validation_0-mlogloss:0.17755
[82]	validation_0-mlogloss:0.17462
[83]	validation_0-mlogloss:0.17216
[84]	validation_0-mlogloss:0.16870
[85]	validation_0-mlogloss:0.16587
[86]	validation_0-mlogloss:0.16335
[87]	validation_0-mlogloss:0.16031
[88]	validation_0-mlogloss:0.15706
[89]	validation_0-mlogloss:0.15435
[90]	validation_0-mlogloss:0.15172
[91]	validation_0-mlogloss:0.14896
[92]	validation_0-ml

[96]	validation_0-mlogloss:0.20305
[97]	validation_0-mlogloss:0.20190
[98]	validation_0-mlogloss:0.20058
[99]	validation_0-mlogloss:0.19956
[100]	validation_0-mlogloss:0.19836
[101]	validation_0-mlogloss:0.19699
[102]	validation_0-mlogloss:0.19499
[103]	validation_0-mlogloss:0.19342
[104]	validation_0-mlogloss:0.19214
[105]	validation_0-mlogloss:0.19061
[106]	validation_0-mlogloss:0.18957
[107]	validation_0-mlogloss:0.18854
[108]	validation_0-mlogloss:0.18747
[109]	validation_0-mlogloss:0.18621
[110]	validation_0-mlogloss:0.18468
[111]	validation_0-mlogloss:0.18391
[112]	validation_0-mlogloss:0.18251
[113]	validation_0-mlogloss:0.18168
[114]	validation_0-mlogloss:0.18100
[115]	validation_0-mlogloss:0.18028
[116]	validation_0-mlogloss:0.17904
[117]	validation_0-mlogloss:0.17811
[118]	validation_0-mlogloss:0.17736
[119]	validation_0-mlogloss:0.17705
[120]	validation_0-mlogloss:0.17632
[121]	validation_0-mlogloss:0.17554
[122]	validation_0-mlogloss:0.17417
[123]	validation_0-mlogloss:0.17

[127]	validation_0-mlogloss:0.10693
[128]	validation_0-mlogloss:0.10627
[129]	validation_0-mlogloss:0.10557
[130]	validation_0-mlogloss:0.10489
[131]	validation_0-mlogloss:0.10471
[132]	validation_0-mlogloss:0.10435
[133]	validation_0-mlogloss:0.10371
[134]	validation_0-mlogloss:0.10345
[135]	validation_0-mlogloss:0.10303
[136]	validation_0-mlogloss:0.10241
[137]	validation_0-mlogloss:0.10173
[138]	validation_0-mlogloss:0.10130
[139]	validation_0-mlogloss:0.10069
[140]	validation_0-mlogloss:0.10038
[141]	validation_0-mlogloss:0.10013
[142]	validation_0-mlogloss:0.09956
[143]	validation_0-mlogloss:0.09913
[144]	validation_0-mlogloss:0.09922
[145]	validation_0-mlogloss:0.09888
[146]	validation_0-mlogloss:0.09835
[147]	validation_0-mlogloss:0.09815
[148]	validation_0-mlogloss:0.09775
[149]	validation_0-mlogloss:0.09738
[150]	validation_0-mlogloss:0.09719
[151]	validation_0-mlogloss:0.09692
[152]	validation_0-mlogloss:0.09655
[153]	validation_0-mlogloss:0.09609
[154]	validation_0-mlogloss:

...

[52]	validation_0-mlogloss:0.15941
[53]	validation_0-mlogloss:0.15535
[54]	validation_0-mlogloss:0.15164
[55]	validation_0-mlogloss:0.14815
[56]	validation_0-mlogloss:0.14498
[57]	validation_0-mlogloss:0.14178
[58]	validation_0-mlogloss:0.13872
[59]	validation_0-mlogloss:0.13586
[60]	validation_0-mlogloss:0.13299
[61]	validation_0-mlogloss:0.13036
[62]	validation_0-mlogloss:0.12779
[63]	validation_0-mlogloss:0.12459
[64]	validation_0-mlogloss:0.12224
[65]	validation_0-mlogloss:0.12001
[66]	validation_0-mlogloss:0.11783
[67]	validation_0-mlogloss:0.11547
[68]	validation_0-mlogloss:0.11255
[69]	validation_0-mlogloss:0.11044
[70]	validation_0-mlogloss:0.10832
[71]	validation_0-mlogloss:0.10639
[72]	validation_0-mlogloss:0.10429
[73]	validation_0-mlogloss:0.10243
[74]	validation_0-mlogloss:0.10031
[75]	validation_0-mlogloss:0.09864
[76]	validation_0-mlogloss:0.09698
[77]	validation_0-mlogloss:0.09528
[78]	validation_0-mlogloss:0.09393
[79]	validation_0-mlogloss:0.09219
[80]	validation_0-ml

[84]	validation_0-mlogloss:0.08247
[85]	validation_0-mlogloss:0.08132
[86]	validation_0-mlogloss:0.07991
[87]	validation_0-mlogloss:0.07865
[88]	validation_0-mlogloss:0.07728
[89]	validation_0-mlogloss:0.07622
[90]	validation_0-mlogloss:0.07516
[91]	validation_0-mlogloss:0.07389
[92]	validation_0-mlogloss:0.07282
[93]	validation_0-mlogloss:0.07176
[94]	validation_0-mlogloss:0.07074
[95]	validation_0-mlogloss:0.06973
[96]	validation_0-mlogloss:0.06879
[97]	validation_0-mlogloss:0.06775
[98]	validation_0-mlogloss:0.06690
[99]	validation_0-mlogloss:0.06602
[100]	validation_0-mlogloss:0.06516
[101]	validation_0-mlogloss:0.06432
[102]	validation_0-mlogloss:0.06364
[103]	validation_0-mlogloss:0.06286
[104]	validation_0-mlogloss:0.06199
[105]	validation_0-mlogloss:0.06130
[106]	validation_0-mlogloss:0.06056
[107]	validation_0-mlogloss:0.06011
[108]	validation_0-mlogloss:0.05951
[109]	validation_0-mlogloss:0.05872
[110]	validation_0-mlogloss:0.05822
[111]	validation_0-mlogloss:0.05773
[112]	va

[116]	validation_0-mlogloss:0.13775
[117]	validation_0-mlogloss:0.13677
[118]	validation_0-mlogloss:0.13615
[119]	validation_0-mlogloss:0.13588
[120]	validation_0-mlogloss:0.13519
[121]	validation_0-mlogloss:0.13414
[122]	validation_0-mlogloss:0.13343
[123]	validation_0-mlogloss:0.13295
[124]	validation_0-mlogloss:0.13210
[125]	validation_0-mlogloss:0.13133
[126]	validation_0-mlogloss:0.13075
[127]	validation_0-mlogloss:0.13000
[128]	validation_0-mlogloss:0.12933
[129]	validation_0-mlogloss:0.12911
[130]	validation_0-mlogloss:0.12832
[131]	validation_0-mlogloss:0.12769
[132]	validation_0-mlogloss:0.12727
[133]	validation_0-mlogloss:0.12712
[134]	validation_0-mlogloss:0.12649
[135]	validation_0-mlogloss:0.12581
[136]	validation_0-mlogloss:0.12583
[137]	validation_0-mlogloss:0.12526
[138]	validation_0-mlogloss:0.12458
[139]	validation_0-mlogloss:0.12407
[140]	validation_0-mlogloss:0.12407
[141]	validation_0-mlogloss:0.12367
[142]	validation_0-mlogloss:0.12352
[143]	validation_0-mlogloss:

[147]	validation_0-mlogloss:0.09053
[148]	validation_0-mlogloss:0.09000
[149]	validation_0-mlogloss:0.08999
[150]	validation_0-mlogloss:0.08982
[151]	validation_0-mlogloss:0.08963
[152]	validation_0-mlogloss:0.08923
[153]	validation_0-mlogloss:0.08924
[154]	validation_0-mlogloss:0.08911
[155]	validation_0-mlogloss:0.08879
[156]	validation_0-mlogloss:0.08884
[157]	validation_0-mlogloss:0.08850
[158]	validation_0-mlogloss:0.08849
[159]	validation_0-mlogloss:0.08818
[160]	validation_0-mlogloss:0.08824
[161]	validation_0-mlogloss:0.08809
[162]	validation_0-mlogloss:0.08771
[163]	validation_0-mlogloss:0.08741
[164]	validation_0-mlogloss:0.08708
[165]	validation_0-mlogloss:0.08686
[166]	validation_0-mlogloss:0.08700
[167]	validation_0-mlogloss:0.08671
[168]	validation_0-mlogloss:0.08627
[169]	validation_0-mlogloss:0.08625
[170]	validation_0-mlogloss:0.08583
[171]	validation_0-mlogloss:0.08579
[172]	validation_0-mlogloss:0.08551
[173]	validation_0-mlogloss:0.08532
[174]	validation_0-mlogloss:

[178]	validation_0-mlogloss:0.05755
[179]	validation_0-mlogloss:0.05745
[180]	validation_0-mlogloss:0.05731
[181]	validation_0-mlogloss:0.05712
[182]	validation_0-mlogloss:0.05722
[183]	validation_0-mlogloss:0.05740
[184]	validation_0-mlogloss:0.05706
[185]	validation_0-mlogloss:0.05683
[186]	validation_0-mlogloss:0.05650
[187]	validation_0-mlogloss:0.05646
[188]	validation_0-mlogloss:0.05621
[189]	validation_0-mlogloss:0.05624
[190]	validation_0-mlogloss:0.05602
[191]	validation_0-mlogloss:0.05590
[192]	validation_0-mlogloss:0.05600
[193]	validation_0-mlogloss:0.05577
[194]	validation_0-mlogloss:0.05573
[195]	validation_0-mlogloss:0.05535
[196]	validation_0-mlogloss:0.05529
[197]	validation_0-mlogloss:0.05514
[198]	validation_0-mlogloss:0.05518
[199]	validation_0-mlogloss:0.05502
[0]	validation_0-mlogloss:1.05664
[1]	validation_0-mlogloss:1.01721
[2]	validation_0-mlogloss:0.98002
[3]	validation_0-mlogloss:0.94382
[4]	validation_0-mlogloss:0.90985
[5]	validation_0-mlogloss:0.87798
[6]	

In [111]:
#to를 위한 파라미터 튜닝
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.25, 
                                                    stratify=train_y, 
                                                    random_state=37)


dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)
dtest_x = xgb.DMatrix(data=test_x)

params = {'max_depth' :6,
          'eta': 0.03, #0.035,0.04, 0.045, 0.05 다 똑같 
          'objective':'multi:softmax',
          'num_class':3,
          'eval_metric':'mlogloss',
          'min_child_weight' : 1,
          'colsample_bytree' : 1,
          'gamma' : 0 #default 0 gamma 0.5 0.95
          }
num_rounds = 500

wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model2 = xgb.train(params=params, 
                      dtrain=dtrain, 
                      num_boost_round=num_rounds, 
                      early_stopping_rounds=100, 
                      evals=wlist)

[0]	train-mlogloss:1.05729	eval-mlogloss:1.06065
[1]	train-mlogloss:1.01811	eval-mlogloss:1.02387
[2]	train-mlogloss:0.98089	eval-mlogloss:0.98962
[3]	train-mlogloss:0.94550	eval-mlogloss:0.95712
[4]	train-mlogloss:0.91180	eval-mlogloss:0.92626
[5]	train-mlogloss:0.87967	eval-mlogloss:0.89606
[6]	train-mlogloss:0.84901	eval-mlogloss:0.86785
[7]	train-mlogloss:0.81972	eval-mlogloss:0.84107
[8]	train-mlogloss:0.79172	eval-mlogloss:0.81485
[9]	train-mlogloss:0.76493	eval-mlogloss:0.79003
[10]	train-mlogloss:0.73927	eval-mlogloss:0.76637
[11]	train-mlogloss:0.71469	eval-mlogloss:0.74303
[12]	train-mlogloss:0.69111	eval-mlogloss:0.72128
[13]	train-mlogloss:0.66850	eval-mlogloss:0.70039
[14]	train-mlogloss:0.64679	eval-mlogloss:0.67979
[15]	train-mlogloss:0.62593	eval-mlogloss:0.66060
[16]	train-mlogloss:0.60589	eval-mlogloss:0.64215
[17]	train-mlogloss:0.58662	eval-mlogloss:0.62411
[18]	train-mlogloss:0.56811	eval-mlogloss:0.60669
[19]	train-mlogloss:0.55027	eval-mlogloss:0.58985
[20]	train

[163]	train-mlogloss:0.02245	eval-mlogloss:0.07149
[164]	train-mlogloss:0.02221	eval-mlogloss:0.07120
[165]	train-mlogloss:0.02199	eval-mlogloss:0.07102
[166]	train-mlogloss:0.02175	eval-mlogloss:0.07088
[167]	train-mlogloss:0.02154	eval-mlogloss:0.07072
[168]	train-mlogloss:0.02130	eval-mlogloss:0.07037
[169]	train-mlogloss:0.02108	eval-mlogloss:0.07016
[170]	train-mlogloss:0.02087	eval-mlogloss:0.06987
[171]	train-mlogloss:0.02066	eval-mlogloss:0.06972
[172]	train-mlogloss:0.02045	eval-mlogloss:0.06943
[173]	train-mlogloss:0.02025	eval-mlogloss:0.06909
[174]	train-mlogloss:0.02003	eval-mlogloss:0.06871
[175]	train-mlogloss:0.01984	eval-mlogloss:0.06838
[176]	train-mlogloss:0.01963	eval-mlogloss:0.06815
[177]	train-mlogloss:0.01945	eval-mlogloss:0.06803
[178]	train-mlogloss:0.01926	eval-mlogloss:0.06765
[179]	train-mlogloss:0.01907	eval-mlogloss:0.06744
[180]	train-mlogloss:0.01889	eval-mlogloss:0.06733
[181]	train-mlogloss:0.01870	eval-mlogloss:0.06706
[182]	train-mlogloss:0.01853	ev

[324]	train-mlogloss:0.00922	eval-mlogloss:0.04904
[325]	train-mlogloss:0.00919	eval-mlogloss:0.04904
[326]	train-mlogloss:0.00917	eval-mlogloss:0.04894
[327]	train-mlogloss:0.00914	eval-mlogloss:0.04899
[328]	train-mlogloss:0.00912	eval-mlogloss:0.04893
[329]	train-mlogloss:0.00910	eval-mlogloss:0.04889
[330]	train-mlogloss:0.00908	eval-mlogloss:0.04872
[331]	train-mlogloss:0.00905	eval-mlogloss:0.04871
[332]	train-mlogloss:0.00904	eval-mlogloss:0.04867
[333]	train-mlogloss:0.00902	eval-mlogloss:0.04865
[334]	train-mlogloss:0.00899	eval-mlogloss:0.04861
[335]	train-mlogloss:0.00897	eval-mlogloss:0.04853
[336]	train-mlogloss:0.00895	eval-mlogloss:0.04846
[337]	train-mlogloss:0.00893	eval-mlogloss:0.04837
[338]	train-mlogloss:0.00890	eval-mlogloss:0.04842
[339]	train-mlogloss:0.00889	eval-mlogloss:0.04825
[340]	train-mlogloss:0.00887	eval-mlogloss:0.04822
[341]	train-mlogloss:0.00884	eval-mlogloss:0.04816
[342]	train-mlogloss:0.00883	eval-mlogloss:0.04812
[343]	train-mlogloss:0.00881	ev

[485]	train-mlogloss:0.00709	eval-mlogloss:0.04254
[486]	train-mlogloss:0.00708	eval-mlogloss:0.04254
[487]	train-mlogloss:0.00708	eval-mlogloss:0.04253
[488]	train-mlogloss:0.00707	eval-mlogloss:0.04252
[489]	train-mlogloss:0.00706	eval-mlogloss:0.04248
[490]	train-mlogloss:0.00705	eval-mlogloss:0.04239
[491]	train-mlogloss:0.00704	eval-mlogloss:0.04237
[492]	train-mlogloss:0.00704	eval-mlogloss:0.04236
[493]	train-mlogloss:0.00703	eval-mlogloss:0.04232
[494]	train-mlogloss:0.00702	eval-mlogloss:0.04232
[495]	train-mlogloss:0.00701	eval-mlogloss:0.04226
[496]	train-mlogloss:0.00700	eval-mlogloss:0.04227
[497]	train-mlogloss:0.00700	eval-mlogloss:0.04220
[498]	train-mlogloss:0.00699	eval-mlogloss:0.04219
[499]	train-mlogloss:0.00698	eval-mlogloss:0.04215


In [112]:
#to 데이터의 score
print(f'best ntree limit은 {xgb_model2.best_ntree_limit}')
y_pred = xgb_model2.predict(dtest, ntree_limit=xgb_model2.best_ntree_limit)
preds = np.round(y_pred).astype(int)
print(f1_score(y_test, preds, average = 'macro'))

best ntree limit은 500
0.9754789272030652


In [ ]:
#to를 위한 파라미터 튜닝
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, 
                                                    test_size=0.25, 
                                                    stratify=train_y, 
                                                    random_state=37)


dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test, label=y_test)
dtest_x = xgb.DMatrix(data=test_x)

params = {'max_depth' : 5,
          'eta': 0.035, #0.035,0.04, 0.045, 0.05 다 똑같 
          'objective':'multi:softmax',
          'num_class':3,
          'eval_metric':'mlogloss',
          'min_child_weight' : 1,
          'colsample_bytree' : 1,
          'gamma' : 1 #default 0 gamma 0.5 0.95
          }
num_rounds = 500

wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model2 = xgb.train(params=params, 
                      dtrain=dtrain, 
                      num_boost_round=num_rounds, 
                      early_stopping_rounds=100, 
                      evals=wlist)

In [ ]:
#to 데이터의 score
print(f'best ntree limit은 {xgb_model2.best_ntree_limit}')
y_pred = xgb_model2.predict(dtest, ntree_limit=xgb_model2.best_ntree_limit)
preds = np.round(y_pred).astype(int)
print(f1_score(y_test, preds, average = 'macro'))

In [113]:
y_preds = xgb_model2.predict(dtest_x, ntree_limit=xgb_model2.best_ntree_limit)
preds_2 = np.round(y_preds).astype(int)
preds2 = pd.DataFrame(preds_2)
p1 = pd.Series(data = p1)
preds20 = pd.concat([preds2, p1], axis = 1)
preds20.columns = ['y_class', 'p1']
preds20.index = preds20['p1']
del preds20['p1']
preds20

,y_class
p1,
0,2
1,2
2,2
15,2
16,2
17,2
18,2
19,2
20,2


In [564]:
preds

array([0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1])

In [80]:
preds10.describe()

,y_class
count,243.0
mean,2.0
std,0.0
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0


In [568]:
preds20.describe()

,y_class
count,243.000000
mean,1.600823
std,0.515383
min,0.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


In [114]:
preds = pd.concat([preds10, preds20], axis = 0)
preds = preds.sort_index(ascending=True)
preds

,y_class
p1,
0,2
1,2
2,2
3,1
4,0
5,1
6,1
7,0
8,0


In [570]:
sub['Y_Class'] = preds['y_class']
sub.to_csv('./t32_XGB_sep_ATOdata_com.csv', index = False)

In [523]:
sub.describe()

,Y_Class
count,310.000000
mean,0.845161
std,0.371160
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [60]:
subm = pd.read_csv('/Users/kimminyoung/Desktop/Dacon_SmartFactory/t30_XGB_Dmatrix_Earlystopping_eta0035_testsize01(0.704).csv')
subm[if p1 is in subm.index]

SyntaxError: invalid syntax (3285705194.py, line 2)

In [12]:
#모델 저장
with open('model_XGB_0683.pickle','wb') as fw:
    pickle.dump(xgb_model, fw)